# check jetson nano

In [1]:
import subprocess
import pkg_resources
import platform
import os, sys
import torch, tensorflow
sys.path.append('../')
from utils.cooling_fan import CoolingFan

In [12]:
cooling_fan = CoolingFan()
print(f'cooling fan speed: {cooling_fan.get_speed()} (max=255)  temperature: {cooling_fan.get_temp()} degree celsius') 

cooling fan speed: 0 (max=255)  temperature: 31.0 degree celsius


In [16]:
# print model
with open('/proc/device-tree/model', 'r') as f:
    print(str(f.read()[:-1]))

NVIDIA Jetson Nano Developer Kit


In [8]:
# network connection(s)
for interface in ['eth0', 'wlan0']:
    operstate = subprocess.check_output('cat /sys/class/net/%s/operstate' % interface, shell=True).decode('ascii')[:-1]
    if operstate == 'up' :  
        cmd = "ifconfig %s | grep -Eo 'inet (addr:)?([0-9]*\.){3}[0-9]*' | grep -Eo '([0-9]*\.){3}[0-9]*' | grep -v '127.0.0.1'" % interface
        ip = subprocess.check_output(cmd, shell=True).decode('ascii')[:-1]
    else:
        ip = ''
    print(f'{interface}\t{operstate}\t{ip}')

eth0	up	192.168.178.92
wlan0	down	


In [9]:
!/usr/local/cuda/bin/nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_21:14:42_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


In [10]:
!cat /usr/include/cudnn_version.h | grep CUDNN

#ifndef CUDNN_VERSION_H_
#define CUDNN_VERSION_H_
#define CUDNN_MAJOR 8
#define CUDNN_MINOR 0
#define CUDNN_PATCHLEVEL 0
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)
#endif /* CUDNN_VERSION_H */


In [11]:
print('python:     ', sys.version.replace('\n', ' '))
print('pytorch:    ', torch.__version__)
print('keras :     ', tensorflow.keras.__version__)
print('tensorflow: ', tensorflow.__version__)
print('CUDNN:      ', torch.backends.cudnn.version())
print('device      ', torch.cuda.get_device_name(torch.cuda.current_device()))
nvidia_jetpack = subprocess.check_output(['apt-cache', 'show', 'nvidia-jetpack']).decode('utf8')
start = nvidia_jetpack.find('Version', 0)
end = nvidia_jetpack.find('\n', start)
print('jetpack     ', nvidia_jetpack[start+9:end])
print(subprocess.check_output(['cat', '/usr/local/cuda/version.txt']).decode('utf8'))

python:      3.6.9 (default, Oct  8 2020, 12:12:24)  [GCC 8.4.0]
pytorch:     1.7.0
keras :      2.4.0
tensorflow:  2.3.1
CUDNN:       8000
device       NVIDIA Tegra X1
jetpack      4.5.1-b17
CUDA Version 10.2.89



In [12]:
# input devices (e.g. to detect devicename of a gamepad)
!ls /dev/input/event*
print('###')
!cat /proc/bus/input/devices

/dev/input/event0  /dev/input/event1  /dev/input/event2
###
I: Bus=0000 Vendor=0000 Product=0000 Version=0000
N: Name="tegra-hda HDMI/DP,pcm=3"
P: Phys=ALSA
S: Sysfs=/devices/70030000.hda/sound/card0/input0
U: Uniq=
E: Enabled=0
H: Handlers=event0 
B: PROP=0
B: EV=21
B: SW=140

I: Bus=0019 Vendor=0001 Product=0001 Version=0100
N: Name="gpio-keys"
P: Phys=gpio-keys/input0
S: Sysfs=/devices/gpio-keys/input/input1
U: Uniq=
E: Enabled=0
H: Handlers=kbd event1 
B: PROP=0
B: EV=3
B: KEY=10000000000000 0

I: Bus=0003 Vendor=046d Product=4024 Version=0111
N: Name="Logitech K400"
P: Phys=usb-70090000.xusb-2.4:1
S: Sysfs=/devices/70090000.xusb/usb1/1-2/1-2.4/1-2.4:1.2/0003:046D:C52B.0003/0003:046D:4024.0004/input/input2
U: Uniq=
E: Enabled=0
H: Handlers=sysrq kbd leds mouse0 event2 
B: PROP=0
B: EV=12001f
B: KEY=3007f 0 0 483ffff17aff32d bf54444600000000 ffff0001 130ff38b17c007 ffff7bfad9415fff febeffdfffefffff fffffffffffffffe
B: REL=1c3
B: ABS=100000000
B: MSC=10
B: LED=1f



In [14]:
# cameras
!ls /dev/video*

/dev/video0  /dev/video1


# jetbot only

In [6]:
# check i2c bus
#  found 0x3c : display Adafruit_SSD1306 
#  found 0x48 : ads1115 -> analog digital converter to get voltages e.g. the 12V on jetbot platine
#  found 0x60 : motor driver : Adafruit_MotorHAT
#  found 0x70 : ?
for i in range(128):
    address = str(hex(i))
    found_address = os.popen("i2cdetect -y -r 1 " + address + " " + address + " | egrep '" + address[2:] + "' | awk '{print $2}'").read()
    if address[2:] in found_address:
        print('found', address)

found 0x3c


In [8]:
# ina219 should return values charging. in my case on this i2c address is nothing found
import jetbot.ina219 as ina219 # import INA219
adress = os.popen("i2cdetect -y -r 1 0x41 0x41 | egrep '41' | awk '{print $2}'").read()
if(adress=='41\n'):
    ina = ina219.INA219(addr=0x41)
else:
    ina = None
    
print(ina)

None


In [36]:
# check power supply on jetbot
# ch 0,1   :  ~9,7 V
# ch 2     :  ~12 V (only if power supply is connected)
# ch 3     :  0 V
# ch 4     :  ~12 V (also if battery only without power supply)
# ch 5,6,7 :  ~2,4V (i2c?)

import jetbot.ads1115 as ads1115 # import ADS1115
adress = os.popen("i2cdetect -y -r 1 0x48 0x48 | egrep '48' | awk '{print $2}'").read()
if(adress=='48\n'):
    ads = ads1115.ADS1115()
    for channel in range(8):
        print(f'channel {channel} {ads.readVoltage(channel)/1000:.2f} volt')
else:
    ads = None